# 分析 OpenACC 代码

此实验面向 C/C++ 程序员。如果您更喜欢使用 Fortran，请单击 [此链接](../Fortran/README.ipynb)

不要忘记查看其他 [OpenACC 资源](https://www.openacc.org/resources) 并加入我们的 [OpenACC Slack 频道](https://www.openacc.org/community#slack)，以分享您的经验并从社区获得更多帮助。

---

让我们通过运行 `nvidia-smi` 命令来执行下面的单元格，以显示有关服务器上运行的 GPU 的信息，该命令随我们将使用的 NVIDIA HPC 软件开发工具包 (SDK) 一起提供。为此，请执行下面的单元格块，方法是将焦点放在它上面（用鼠标单击它），然后按 Ctrl-Enter，或按上面工具栏中的播放按钮。如果一切顺利，您应该会看到灰色单元格下方返回一些输出。

In [ ]:
!nvidia-smi

---
## 简介

我们这个实验的目标是了解代码分析到底是什么，以及如何使用它来帮助我们编写强大的并行程序。

<img src="../images/development-cycle.png" width="70%" height="70%">

这是 OpenACC 3 步开发周期。

**分析**您的代码以确定最有可能需要并行化或优化的地方。

从最耗时的部分开始**并行化**您的代码并检查其正确性。

**优化**您的代码以提高并行化带来的加速效果。

我们目前正在处理**分析**步骤。 在进入接下来的两个步骤之前，我们将使用 Nsight Systems 分析器来了解一个相对简单的示例代码。

---

## 运行代码

分析此代码的第一步是运行它。我们需要在进行任何更改之前记录程序的结果，以便稍后将其与并行代码的结果进行比较。记录程序运行所需的时间也很重要，因为这将是我们判断并行化是否能提高性能的主要指标。

### 编译代码

我们使用 NVIDIA HPC SDK C/C++ 和 Fortran 编译器来编译我们的代码。您不需要记住编译器命令来完成本实验，但是，如果您想将自己的个人代码与 OpenACC 并行化，了解这些命令会很有帮助。

**nvc：** 这是编译 C 代码的命令

**nvc++：** 这是编译 C++ 代码的命令

**nvfortran：** 这是编译 Fortran 代码的命令

**-fast：** 此编译器标志将允许编译器对我们的代码执行额外的优化

In [ ]:
!nvc  -fast -o laplace -Mprof=ccff -I/opt/nvidia/hpc_sdk/Linux_x86_64/23.9/cuda/12.2/include jacobi.c laplace2d.c && echo "Compilation Successful!" && ./laplace

### 理解代码结果

当我们分析代码时，程序的输出将更有意义。要记住的最重要的事情是，我们需要这些输出值保持一致。如果这些输出在我们并行化代码的任何时候发生变化，我们就知道我们犯了一个错误。为简单起见，请关注最后一个输出，它发生在迭代 900 时。记录程序运行所需的时间也很有帮助。我们在并行化代码时的目标最终是使其更快，因此我们需要知道我们的“基本运行时”，以便知道代码是否运行得更快。

---

## 分析代码

现在我们知道了代码运行的时间以及代码的输出，我们应该能够查看代码并大致了解正在发生的事情。代码包含在两个文件中，您可以打开并查看。

[jacobi.c](jacobi.c)

[laplace2d.c](laplace2d.c)

您可以自己阅读这两个文件，但我们也会在下面的“代码分解”中重点介绍最重要的部分。

### 代码说明

代码模拟了二维金属板上的热量分布。一开始，金属板不会受热，这意味着整个金属板的温度都是室温。然后，对金属板的边缘施加恒定的热量，然后代码将模拟热量在金属板上的分布。

这是模拟开始前金属板的视觉表示：

<img src="../images/plate1.png" width="70%" height="70%">

我们可以看到，除了顶部边缘外，金属板的温度均匀地为室温。在 [laplace2d.c](../C/laplace2d.c) 文件中，我们看到一个名为 `initialize` 的子程序。这个函数就是“加热”金属板顶部边缘的函数。
  
```cpp
void initialize(double *restrict A, double *restrict Anew, int m, int n)  
{  
    memset(A, 0, n * m * sizeof(double));  
    memset(Anew, 0, n * m * sizeof(double));  
  
    for(int i = 0; i < m; i++){  
        A[i] = 1.0;  
        Anew[i] = 1.0;  
    }  
}  
```

加热顶部边缘后，代码将模拟热量沿板的长度分布。随着模拟的进行，我们将保持顶部边缘的恒定热量。

这是我们模拟几次迭代后的板：

<img src="../images/plate2.png" width="70%" height="70%">

这就是理论：简单的热量分布。然而，我们更感兴趣的是代码的工作原理。

### 代码分解

二维板由包含双精度浮点值的二维数组表示。这些双精度浮点值表示温度；0.0 表示室温，1.0 表示最高温度。二维板有两种状态，一种表示当前温度，一种表示模拟下一步的预期温度值。这两种状态分别由数组 **`A`** 和 **`Anew`** 表示。以下是这些数组的直观表示，顶部边缘“加热”。

![plate_sim2.png](../images/plate_sim2.png)  
    
在两个数组中模拟此状态对于我们的 **`calcNext`** 函数非常重要。我们的 calcNext 本质上是我们的“模拟”函数。calcNext 将查看 A 的内部元素（即除板边缘之外的所有元素），并根据其相邻元素的温度更新每个元素的温度。如果我们尝试就地计算（仅使用 **`A`**），则每个元素都会根据先前元素的更新温度计算其新温度。这种数据依赖性不仅会阻止代码并行化，而且还会在串行运行时导致不正确的结果。通过计算临时数组 **`Anew`**，我们确保在更新 **`A`** 数组之前已完成模拟的整个步骤。

![plate_sim3.png](../images/plate_sim3.png)  

下面是 `calcNext` 函数：

```cpp
01 double calcNext(double *restrict A, double *restrict Anew, int m, int n)
02 {
03     double error = 0.0;  
04     for( int j = 1; j < n-1; j++)  
05     {  
06        for( int i = 1; i < m-1; i++ )   
07        {  
08            Anew[OFFSET(j, i, m)] = 0.25 * ( A[OFFSET(j, i+1, m)] + A[OFFSET(j, i-1, m)]  
09                                           + A[OFFSET(j-1, i, m)] + A[OFFSET(j+1, i, m)]);  
10            error = fmax( error, fabs(Anew[OFFSET(j, i, m)] - A[OFFSET(j, i , m)]));  
11        }  
12    }  
13    return error;  
14 }  
```

我们看到，在第 07 行和第 08 行中，我们通过平均相邻元素的当前值来计算 `i,j` 处 `Anew` 的值。第 09 行是我们通过查看 `i,j` 元素在此步骤中的变化量并找到此 `error` 的最大值来计算模拟的当前变化率。如果在完成最大迭代次数之前模拟达到稳定状态，这使我们能够短路模拟。

最后，我们的 `swap` 函数会将 `Anew` 的内容复制到 `A` 。

```cpp
01 void swap(double *restrict A, double *restrict Anew, int m, int n)
02 {	
03    for( int j = 1; j < n-1; j++)
04    {
05        for( int i = 1; i < m-1; i++ )
06        {
07            A[OFFSET(j, i, m)] = Anew[OFFSET(j, i, m)];    
08        }
09    }
10 }
```

---

<a name="profilecode"></a>
## 分析代码

现在您应该对代码的作用有了很好的了解。如果没有，请花一点时间阅读前面的部分，确保您在继续之前理解代码。现在是时候分析代码以更好地了解应用程序在运行时花费的时间了。为了分析我们的代码，我们将使用 Nsight Systems。

Nsight Systems 工具提供系统范围的性能分析，以便可视化应用程序的算法，帮助识别优化机会，并提高在由多个 CPU 和 GPU 组成的系统上运行的应用程序的性能。

#### Nsight Systems 时间轴
- CPU 行有助于定位 CPU 核心的空闲时间。每行显示进程的线程如何利用 CPU 核心。
<img src="../images/cpu.png" width="80%" height="80%">

- 线程行显示每个线程活动的详细视图，包括操作系统运行时库使用情况、CUDA API 调用、NVTX 时间范围和事件（如果集成在应用程序中）。
<img src="../images/thread.png" width="80%" height="80%">

- CUDA 工作负载行显示内核和内存传输活动。
<img src="../images/cuda.png" width="80%" height="80%">

### 使用命令行界面进行分析
要分析您的应用程序，您可以使用图形用户界面 (GUI) 或命令行界面 (CLI)。在此实验中，我们将使用 CLI 为迷你应用程序生成分析器报告，并通过 GUI 查看它。

Nsight Systems 命令行界面名为 `nsys` 。以下是典型的命令行调用：

`nsys profile -t nvtx --stats=true --force-overwrite true -o laplace ./laplace`

其中，本实验使用的命令开关选项为：
- `profile`：启动分析会话
- `-t`：选择要跟踪的 API（本例中为 nvtx 和 openacc）
- `--stats`：如果为真，则在收集后生成统计摘要
- `--force-overwrite`：如果为真，则覆盖现有的生成报告
- `-o`：在收集结束时创建的中间结果文件的名称（.qdrep 文件名）

**注意**：您无需记住分析器选项。您始终可以从命令行运行`nsys --help`或`nsys [特定命令] --help`，并使用必要的选项或分析器参数。
有关 Nsight 分析器和 NVTX 的更多信息，请参阅 [分析器文档](https://docs.nvidia.com/nsight-systems/) 。

<a name="viewreport"></a>
### 如何查看报告
使用 CLI 分析应用程序时，有两种方法可以查看分析器的报告。

1) 在终端上使用 `--stats` 选项：通过使用 `--stats` 开关选项，在收集分析数据后，分析结果将显示在控制台终端上。

<img src="../images/laplas3.png" width="100%" height="100%">

2) NVIDIA Nsight System GUI：分析会话结束后，将创建一个 `*.qdrep` 文件。可以使用 *File -> Open* 将该文件加载到 Nsight Systems GUI 中。如果您想在本地机器上查看此内容，则要求本地系统安装了相同版本的 CUDA 工具包，并且 Nsight System GUI 版本应与 CLI 版本匹配。

**注意**：为了能够看到 Nsight System 分析器输出，请从[此处](https://developer.nvidia.com/nsight-systems) 下载 Nsight System 最新版本。

要查看分析器报告，只需从 GUI 打开文件（File > Open）。

<img src="../images/nsight_open.png" width="80%" height="80%">

### 使用 NVIDIA 工具扩展 (NVTX)
NVIDIA 工具扩展 (NVTX) 是一个基于 C 的应用程序编程接口 (API)，用于注释应用程序中的事件、时间范围和资源。 NVTX 将分析应用程序的逻辑带入 Profiler，使 Profiler 显示的数据更易于分析，并能够将显示的数据与分析应用程序的操作相关联。

在此实验中，我们使用 Nsight Systems 命令行界面分析应用程序并收集时间线。我们还将跟踪 NVTX API（已集成到应用程序中）。

NVTX 工具是一种强大的机制，允许用户手动检测其应用程序。然后，NVIDIA Nsight Systems 可以收集信息并将其显示在时间线上。它对于跟踪 CPU 事件和时间范围特别有用，并大大提高了时间线的可读性。

**如何使用 NVTX**：对于 C 代码，在源代码中添加 `#include "nvtx3/nvToolsExt.h"` ，并将要捕获事件的代码部分包装到对 NVTX API 函数的调用中。例如，尝试在 `main()` 函数的开头添加 `nvtxRangePush("main")` ，并在最后的 return 语句之前添加 `nvtxRangePop()`。

下面的示例代码片段展示了范围事件的使用。可以在 Nsight Systems 时间线视图中查看生成的 NVTX 标记。

```cpp
    nvtxRangePushA("init");
    initialize(A, Anew, m, n);
    nvtxRangePop();

    printf("Jacobi relaxation Calculation: %d x %d mesh\n", n, m);

    double st = omp_get_wtime();
    int iter = 0;

    nvtxRangePushA("while");
    while ( error > tol && iter < iter_max )
    {
        nvtxRangePushA("calc");
        error = calcNext(A, Anew, m, n);
        nvtxRangePop();

        nvtxRangePushA("swap");
        swap(A, Anew, m, n);
        nvtxRangePop();

        if(iter % 100 == 0) printf("%5d, %0.6f\n", iter, error);

        iter++;
    }
    nvtxRangePop();
   
```

<img src="../images/nvtx.PNG" width="80%" height="80%">

详细的 NVTX 文档可在 [CUDA Profiler 用户指南](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvtx) 下找到。

我们将首先使用命令行选项分析我们之前创建的 laplace 可执行文件。运行 `nsys` 命令：

In [ ]:
!nsys profile -t nvtx --stats=true --force-overwrite true -o laplace ./laplace

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并 <mark>右键单击</mark> [此处](laplace.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看分析器报告，请参阅 [如何查看报告](#viewreport) 部分。

<img src="../images/nsys2_devbox.png" width="100%" height="100%">

如果我们放大，我们可以看到代码的每个部分运行所花费的时间。这些部分已用 NVTX 标记，可以在时间线视图的 NVTX 行上看到。此信息很重要，因为它使我们能够做出明智的决定，确定首先优化代码的哪些部分。为了物有所值，我们希望专注于代码中最耗时的部分。

接下来，我们将编译、运行和分析代码的并行版本，并分析差异。

### 可选 - __c_mcopy8 来自哪里？

当我们之前编译代码时，我们忽略了任何类型的编译器反馈。事实证明，即使是顺序代码，编译器也会执行大量优化。如果您使用 `-Minfo=opt` 标志再次编译代码，该标志指示编译器打印有关如何优化代码的其他信息，那么这个奇怪的例程的来源就会变得更加明显。之后，您应该会看到 `__c_mcopy8` 实际上是应用于 **`swap`** 函数的优化。请注意，在下面的输出中，在 `laplace2d.c` 的第 64 行（发生在 `swap` 例程内），编译器确定我们的循环正在执行内存复制，它认为可以通过调用 `__c_mcopy8` 函数更有效地执行。

```cpp
laplace2d.c:
swap:
     53, Memory copy idiom, loop replaced by call to __c_mcopy8
```

In [ ]:
!nvc -fast -Minfo=opt -I/opt/nvidia/hpc_sdk/Linux_x86_64/23.9/cuda/12.2/include -o laplace jacobi.c laplace2d.c

---

## 在多核 CPU 上运行我们的并行代码

在未来的实验中，您将并行化代码以在多核 CPU 上运行。这是最简单的起点，因为它不需要我们考虑在不同的内存之间复制数据。为了让您能够在多核 CPU 上使用 Nsight Systems 进行分析，我们提供了代码的并行版本。您将能够在下一个实验中自己并行化代码。

In [ ]:
!nvc -fast -ta=multicore -Minfo=accel -I/opt/nvidia/hpc_sdk/Linux_x86_64/23.9/cuda/12.2/include -o laplace_parallel ./solutions/parallel/jacobi.c ./solutions/parallel/laplace2d.c && ./laplace_parallel

### 编译多核代码

同样，您不需要记住编译器命令即可完成本实验。但是，如果您想将 OpenACC 与您自己的个人代码一起使用，您将需要学习它们。

**-ta** ：此标志将告诉编译器为特定的并行硬件编译我们的代码。TA 代表**“目标加速器”**，加速器是任何加速性能的设备（在我们的例子中，这意味着并行硬件。）省略 -ta 标志将导致代码按顺序编译。
**-ta=multicore** 将告诉编译器专门为多核 CPU 并行化代码。
**-Minfo** ：此标志将告诉编译器在编译代码时给我们一些反馈。
**-Minfo=accel** ：只会给我们关于代码并行化的反馈。
**-Minfo=opt** ：将给我们关于顺序优化的反馈。
**-Minfo=all** ：将提供所有反馈；这包括有关并行化、顺序优化甚至由于某种原因无法优化的代码部分的反馈。

如果您想查看之前的 c_mcopy8，请尝试使用 `-Minfo=accel,opt` 切换 Minfo 标志。您可以使用环境变量 `ACC_NUM_CORES` 来控制程序将使用多少个线程来运行并行计算区域。默认值是计算系统上有多少个可用的核心。

---

## 分析多核代码

现在，让我们分析应用程序并检查 NVTX 行。

In [ ]:
!nsys profile -t nvtx --stats=true --force-overwrite true -o laplace_parallel ./laplace_parallel

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并 <mark>右键单击</mark> [此处](laplace_parallel.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它并查看时间线视图上的 NVTX 行。要在本地查看分析器报告，请参阅 [如何查看报告](#viewreport) 部分。

<img src="../images/nsys_parallel2_devbox.png" width="100%" height="100%">

如您所见，由于我们现在可以通过将工作分散到多个线程来并行执行部分代码，因此应用程序运行时间减少了。

---

## 总结

现在，我们对程序的运行方式以及程序的哪些部分耗时有了很好的了解。在下一个实验中，我们将使用 OpenACC 并行化我们的程序。

我们正在处理一个非常简单的代码，专门用于教学目的。这意味着，就复杂性而言，它可能相当平淡无奇。如果您选择处理“真实世界”代码，那么分析代码将变得更加有用；该代码可能包含数百个函数或数百万行代码。当我们只有 4 个函数并且我们的整个代码仅包含在两个文件中时，分析可能看起来微不足道，但是，在并行化真实世界代码时，分析将是您最大的资产之一。

---

## 额外任务

目前，我们专注于多核 CPU。最终，我们将过渡到 GPU。如果您熟悉 GPU，并且想使用 GPU 配置文件，那么请随意尝试此奖励任务。如果您现在不想完成此任务，您将有机会在以后的实验中完成（我们还将在其中详细解释正在发生的事情。）

运行此脚本以在 GPU 上编译/运行我们的代码，然后对其进行配置文件。

In [ ]:
!nvc -fast -ta=tesla -Minfo=accel -I/opt/nvidia/hpc_sdk/Linux_x86_64/23.9/cuda/12.2/include -o laplace_gpu ./solutions/gpu/jacobi.c ./solutions/gpu/laplace2d.c && ./laplace_gpu

In [ ]:
!nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o laplace_gpu ./laplace_gpu

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并 <mark>右键单击</mark> [此处](laplace_gpu.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它并与以前的报告进行比较。要在本地查看分析器报告，请参阅 [如何查看报告](#viewreport) 部分。

<img src="../images/nsys_gpu_v100_c.png" width="100%" height="100%">

展开时间线视图上的 GPU 行以查看内核和内存移动。

<img src="../images/nsys_gpu2_v100_c.png" width="100%" height="100%">

分析愉快！

---

## 实验后总结

如果您想下载此实验以供以后查看，建议您转到浏览器的文件菜单（而不是 Jupyter 笔记本文件菜单）并保存完整的网页。这将确保图像也被复制下来。

您还可以执行以下单元块来创建您正在处理的文件的 zip 文件，并使用下面的链接下载它。

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**执行上述 zip 命令后**，您应该能够通过按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](openacc_files.zip) 并选择*将链接另存为*来下载并保存 zip 文件。

# 许可

本材料由 NVIDIA Corporation 根据知识共享署名 4.0 国际 (CC BY 4.0) 发布。